In [ ]:
import torchvision.models as models
import torch.nn as nn
from torchinfo import summary
import torch.optim
from tqdm import tqdm
import os
from torch.utils.data import Dataset, DataLoader
import json
from glob import glob
from PIL import Image
import torchvision.transforms.functional as F
from torchvision import transforms
import numpy as np
import numbers
import time
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import random
import csv
import seaborn as sn
import yaml
import pandas as pd
import datetime
import statistics
import itertools
from colorama import Fore, Back, Style
from borb.pdf import Document, X11Color, Page, Alignment, SingleColumnLayoutWithOverflow, Paragraph, PDF, TableCell, FlexibleColumnWidthTable
from borb.pdf.canvas.layout.annotation.remote_go_to_annotation import RemoteGoToAnnotation
from decimal import Decimal
from IPython.display import clear_output

# Paramètres généraux

In [ ]:
data_dir = 'dataset' # name of dataset (with subfolder 'train', 'val', 'test')
grid_results_dir = 'Grid_Results' # name of main directory with results fo grd schearch

# Paramètres de grille de recherche

In [ ]:
results_pdf = 'results_SGD.pdf'
# grille optimiseurs
optimizers = ['SGD']                        # in 'SGD', 'Adadelta', 'Adam', 'AdamW', 'NAdam', 'RAdam', 'RMSprop', ('LBFGS', 'Adagrad' error)
lrs = [1, 0.1, 0.01,0.001, 1e-4]            # float: All
weight_decays = [0, 0.01]                   # float: All other than LBFGS
epsilons = [1e-06, 1e-07]                   # float: Adadelta, Adagrad, Adam, AdamW, NAdam, RAdam, RMSprop
rhos = [0.9, 0.95]                          # float: Adadelta, Adagrad
initial_accumulator_values = [0.01]         # Adagrad
betass = [[0.9, 0.999], [0.85, 0.95]]       # [float, float]: Adam, AdamW, NAdam, RAdam
max_iters = [20]                            # int: LBFGS
momentum_decays = [1e-4]                    # float: NAdam
alphas = [0]                                # float: RMSprop
momentums = [0, 0.9]                        # float: SGD, RMSprop

# grille loss function
loss_functions = ['MAE', 'MSE', 'Huber']    # in 'MAE', 'MSE', 'Huber'
deltas = [1, 1.35, 2, 3]                    # float: Huber

# autres hyper paramètres
imgszs = [224]                              # int or [int, int]: (w,h), minimum 224 (pour resnet)
batch_sizes = [64]                          # int: maximum 256 avec resnet18 modifier avec 3 couches FC
num_epochss = [100]                         # int
freezes = [False]                           # bool
earlystop = [25]                            # None ou int

In [ ]:
def mult(L):
    """Fait le produit de la taille des éléments de la liste

    Args:
        L (list: list): Liste de listes de paramètres

    Returns:
        int: Nombre de combinaisons possibles pour cette liste de listes
    """
    res = 1
    for l in L:
        res*=len(l)
    return res

In [ ]:
def combinaisons():
    """Donne le nombre de combinaisons qui serons testées

    Returns:
        int: Nombre de combinaisons
    """
    nb_combinaisons = len(freezes)*len(batch_sizes)*len(imgszs)*len(num_epochss)
    if 'Huber' in loss_functions:
        nb_combinaisons *= len(loss_functions)-1+len(deltas)
    nb_combinaisons_crit = 0
    for crit in optimizers:
        match crit:
            case 'SGD':
                nb_combinaisons_crit += mult(list((lrs, weight_decays, momentums)))
            case 'Adadelta':
                nb_combinaisons_crit += mult(list((lrs, weight_decays, epsilons, rhos)))
            case 'Adagrad':
                nb_combinaisons_crit += mult(list((lrs, weight_decays, epsilons, initial_accumulator_values)))
            case 'RMSprop':
                nb_combinaisons_crit += mult(list((lrs, weight_decays, epsilons, alphas, momentums)))
            case 'Adam':
                nb_combinaisons_crit += mult(list((lrs, weight_decays, epsilons, betass)))
            case 'AdamW':
                nb_combinaisons_crit += mult(list((lrs, weight_decays, epsilons, betass)))
            case 'LBFGS':
                nb_combinaisons_crit += mult(list((lrs, max_iters)))
            case 'NAdam':
                nb_combinaisons_crit += mult(list((lrs, weight_decays, epsilons, betass, momentum_decays)))
            case 'RAdam':
                nb_combinaisons_crit += mult(list((lrs, weight_decays, epsilons, betass)))
    nb_combinaisons *=nb_combinaisons_crit
    return nb_combinaisons

# Fonctions

In [ ]:
def get_opt_params_sets(optimizer):
    """Donne un liste des combinainsons de paramètres possibles pour cette optimiseur

    Args:
        optimizer (str): Optimiseur

    Returns:
        list: Liste de listes de paramètres d'optimiseur
    """
    match optimizer:
        case 'SGD':
            params_sets = list(itertools.product(lrs, weight_decays, momentums))
        case 'Adadelta':
            params_sets = list(itertools.product(lrs, weight_decays, epsilons, rhos))
        case 'Adagrad':
            params_sets = list(itertools.product(lrs, weight_decays, epsilons, initial_accumulator_values))
        case 'RMSprop':
            params_sets = list(itertools.product(lrs, weight_decays, epsilons, alphas, momentums))
        case 'Adam':
            params_sets = list(itertools.product(lrs, weight_decays, epsilons, betass))
        case 'AdamW':
            params_sets = list(itertools.product(lrs, weight_decays, epsilons, betass))
        case 'LBFGS':
            params_sets = list(itertools.product(lrs, max_iters))
        case 'NAdam':
            params_sets = list(itertools.product(lrs, weight_decays, epsilons, betass, momentum_decays))
        case 'RAdam':
            params_sets = list(itertools.product(lrs, weight_decays, epsilons, betass))
    return list(map(list, params_sets))
        


In [ ]:
def get_hyper_params_sets(loss_function, optimizer):
    """Donne une liste de combinaisons d'hyperparamètres pour la fonction de perte et l'optimiseur

    Args:
        loss_function (str): Fonction de perte
        optimizer (str): Optimiseur

    Returns:
        list: Liste de listes de paramètres pour la function de perte et l'optimiseur
    """
    optim_sets = get_opt_params_sets(optimizer)
    if loss_function == 'Huber':
        params_sets = list(itertools.product(deltas, optim_sets))
        params_sets = list(map(list, params_sets))
        params_sets = [[xs[0]]+[x for x in xs[1]] for xs in params_sets]
        return params_sets
    return list(map(list, optim_sets))


In [ ]:
def get_hyper_params_dict(results_dir, hyper_params_set):
    """Donne un dictionnaire correspondant a une combinaison d'hyperparamètres

    Args:
        results_dir (str): Chemin d'accès au dossier de résultats
        hyper_params_set (list): Liste des hyperparamètres (la taille change celon la fonction de perte et l'optimiseur dans la combinaison qui n'ont pas tous les mêmes hyperparamètres)

    Returns:
        dict: Dictionnaire avec les hyperparamètres
    """
    hyper_params_dict = {
        'results_dir': results_dir,
        'earlystop': hyper_params_set[0],
        'freeze': hyper_params_set[1],
        'batch_size': hyper_params_set[3],
        'epochs': hyper_params_set[4],
        'loss_function': hyper_params_set[5],
        'optimizer': hyper_params_set[6],
        'dataset': data_dir
    }
    if isinstance(hyper_params_set[2],int):
        imgsz = [hyper_params_set[2],hyper_params_set[2]]
    else:
        imgsz = hyper_params_set[2]
    hyper_params_dict['imgsz'] = imgsz
    if hyper_params_set[5] == 'Huber':
        hyper_params_dict['delta'] = hyper_params_set[7]
        d = 1
    else:
        hyper_params_dict['delta'] = None
        d = 0
    match hyper_params_set[6]:
        case 'SGD':
            hyper_params_dict['lr'] = hyper_params_set[7+d]
            hyper_params_dict['weight_decay'] = hyper_params_set[8+d]
            hyper_params_dict['epsilon'] = None
            hyper_params_dict['rho'] = None
            hyper_params_dict['initial_accumulator_value'] = None
            hyper_params_dict['betas'] = None
            hyper_params_dict['max_iter'] = None
            hyper_params_dict['momentum_decay'] = None
            hyper_params_dict['alpha'] = None
            hyper_params_dict['momentum'] = hyper_params_set[9+d]
        case 'Adadelta':
            hyper_params_dict['lr'] = hyper_params_set[7+d]
            hyper_params_dict['weight_decay'] = hyper_params_set[8+d]
            hyper_params_dict['epsilon'] = hyper_params_set[9+d]
            hyper_params_dict['rho'] = hyper_params_set[10+d]
            hyper_params_dict['initial_accumulator_value'] = None
            hyper_params_dict['betas'] = None
            hyper_params_dict['max_iter'] = None
            hyper_params_dict['momentum_decay'] = None
            hyper_params_dict['alpha'] = None
            hyper_params_dict['momentum'] = None
        case 'Adagrad':
            hyper_params_dict['lr'] = hyper_params_set[7+d]
            hyper_params_dict['weight_decay'] = hyper_params_set[8+d]
            hyper_params_dict['epsilon'] = hyper_params_set[9+d]
            hyper_params_dict['rho'] = None
            hyper_params_dict['initial_accumulator_value'] = hyper_params_set[10+d]
            hyper_params_dict['betas'] = None
            hyper_params_dict['max_iter'] = None
            hyper_params_dict['momentum_decay'] = None
            hyper_params_dict['alpha'] = None
            hyper_params_dict['momentum'] = None
        case 'RMSprop':
            hyper_params_dict['lr'] = hyper_params_set[7+d]
            hyper_params_dict['weight_decay'] = hyper_params_set[8+d]
            hyper_params_dict['epsilon'] = hyper_params_set[9+d]
            hyper_params_dict['rho'] = None
            hyper_params_dict['initial_accumulator_value'] = None
            hyper_params_dict['betas'] = None
            hyper_params_dict['max_iter'] = None
            hyper_params_dict['momentum_decay'] = None
            hyper_params_dict['alpha'] = hyper_params_set[10+d]
            hyper_params_dict['momentum'] = hyper_params_set[11+d]
        case 'Adam':
            hyper_params_dict['lr'] = hyper_params_set[7+d]
            hyper_params_dict['weight_decay'] = hyper_params_set[8+d]
            hyper_params_dict['epsilon'] = hyper_params_set[9+d]
            hyper_params_dict['rho'] = None
            hyper_params_dict['initial_accumulator_value'] = None
            hyper_params_dict['betas'] = hyper_params_set[10+d]
            hyper_params_dict['max_iter'] = None
            hyper_params_dict['momentum_decay'] = None
            hyper_params_dict['alpha'] = None
            hyper_params_dict['momentum'] = None
        case 'AdamW':
            hyper_params_dict['lr'] = hyper_params_set[7+d]
            hyper_params_dict['weight_decay'] = hyper_params_set[8+d]
            hyper_params_dict['epsilon'] = hyper_params_set[9+d]
            hyper_params_dict['rho'] = None
            hyper_params_dict['initial_accumulator_value'] = None
            hyper_params_dict['betas'] = hyper_params_set[10+d]
            hyper_params_dict['max_iter'] = None
            hyper_params_dict['momentum_decay'] = None
            hyper_params_dict['alpha'] = None
            hyper_params_dict['momentum'] = None
        case 'LBFGS':
            hyper_params_dict['lr'] = hyper_params_set[7+d]
            hyper_params_dict['weight_decay'] = None
            hyper_params_dict['epsilon'] = None
            hyper_params_dict['rho'] = None
            hyper_params_dict['initial_accumulator_value'] = None
            hyper_params_dict['betas'] = None
            hyper_params_dict['max_iter'] = hyper_params_set[8+d]
            hyper_params_dict['momentum_decay'] = None
            hyper_params_dict['alpha'] = None
            hyper_params_dict['momentum'] = None
        case 'NAdam':
            hyper_params_dict['lr'] = hyper_params_set[7+d]
            hyper_params_dict['weight_decay'] = hyper_params_set[8+d]
            hyper_params_dict['epsilon'] = hyper_params_set[9+d]
            hyper_params_dict['rho'] = None
            hyper_params_dict['initial_accumulator_value'] = None
            hyper_params_dict['betas'] = hyper_params_set[10+d]
            hyper_params_dict['max_iter'] = None
            hyper_params_dict['momentum_decay'] = hyper_params_set[11+d]
            hyper_params_dict['alpha'] = None
            hyper_params_dict['momentum'] = None
        case 'RAdam':
            hyper_params_dict['lr'] = hyper_params_set[7+d]
            hyper_params_dict['weight_decay'] = hyper_params_set[8+d]
            hyper_params_dict['epsilon'] = hyper_params_set[9+d]
            hyper_params_dict['rho'] = None
            hyper_params_dict['initial_accumulator_value'] = None
            hyper_params_dict['betas'] = hyper_params_set[10+d]
            hyper_params_dict['max_iter'] = None
            hyper_params_dict['momentum_decay'] = None
            hyper_params_dict['alpha'] = None
            hyper_params_dict['momentum'] = None
    return hyper_params_dict

In [ ]:
def make_grid():
    """Génère la grille de recherche

    Returns:
        list: Liste de dictionnaire correspondants aux combinaisons à tester
    """
    grid = []
    with tqdm(total=combinaisons(), desc = 'préparation', colour='green')as pbar:
        hight_hyper_params = list(itertools.product(earlystop, freezes, imgszs, batch_sizes, num_epochss, loss_functions, optimizers))
        hight_hyper_params = list(map(list, hight_hyper_params))
        for hhp in hight_hyper_params:
            low_hyper_params_sets = get_hyper_params_sets(hhp[5], hhp[6])
            hyper_params_sets = list()
            for low_hyper_params_set in low_hyper_params_sets:
                hyper_params_sets.append([*hhp, *low_hyper_params_set])
            for hyper_params_set in hyper_params_sets:
                results_dir = grid_results_dir
                #earlystop
                results_dir = os.path.join(results_dir, f'earlystop_{hyper_params_set[0]}')
                # freeze
                if hyper_params_set[1]:
                    results_dir = os.path.join(results_dir, 'freeze')
                else:
                    results_dir = os.path.join(results_dir, 'no_freeze')
                # image size
                if isinstance(hyper_params_set[2], int):
                    imgsz_dir = 'imgsz_' + str(hyper_params_set[2]) + '-' + str(hyper_params_set[2])
                else:
                    imgsz_dir = 'imgsz_' + str(hyper_params_set[2][0]) + '-' + str(hyper_params_set[2][1])
                results_dir = os.path.join(results_dir, imgsz_dir)
                # nom batch
                batch_dir = 'batch_' + str(hyper_params_set[3])
                # nom epochs
                epochs_dir = 'epochs_' + str(hyper_params_set[4])
                # join batch, epochs, criterion
                results_dir = os.path.join(results_dir, batch_dir, epochs_dir, hyper_params_set[5])
                # ajoute subfolder si huber loss
                if hyper_params_set[5] == 'Huber':
                    delta_dir = 'delta_' + str(hyper_params_set[7])
                    results_dir = os.path.join(results_dir, delta_dir)
                # optimiseur
                results_dir = os.path.join(results_dir, hyper_params_set[6])
                # lr
                results_dir = os.path.join(results_dir, 'lr_{}'.format(hyper_params_set[8 if hyper_params_set[5] == 'Huber' else 7]))
                #low hyp name
                low_dir = 'hyp'
                for i in range(9 if hyper_params_set[5] == 'Huber' else 8, len(hyper_params_set)):
                    if isinstance(hyper_params_set[i],(int, float, str)):
                        low_dir += '_' + str(hyper_params_set[i])
                    else:
                        low_dir += '_' + str(hyper_params_set[i][0])+'-'+str(hyper_params_set[i][1])
                results_dir = os.path.join(results_dir, low_dir)
                hyper_params_dict = get_hyper_params_dict(results_dir, hyper_params_set)
                grid.append(hyper_params_dict)
                pbar.update(1)
    return grid


In [ ]:
def gen_new_model(freeze):
    """Génère un nouveau modèles de regression

    Args:
        freeze (bool): Booléen pour savoir si le modèle doit figer ses poids (sauf les nouvelles couches modifiées)

    Returns:
        nn.Module: Modèle de regression
    """
    model = models.resnet18(weights='IMAGENET1K_V1')

    #modification de la dernière couche
    model.fc = nn.Sequential( 
        nn.Linear(512, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Linear(1024, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Linear(1024, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 1)
    )

    # gel ou non du reste
    if freeze:
        for param in model.parameters():
            param.requires_grad = False

        for param in model.fc.parameters():
            param.requires_grad = True
    return model


In [ ]:
def parse_json_list(path):
    """Analyse le fichier d'annotations

    Args:
        path (str): Chemin vers le fichier d'annotations (json)

    Returns:
        list: Liste d'entier correspondants aux valeurs des annotations
    """
    fname = glob('*.json', dir_fd=path)[0]
    with open(os.path.join(path, fname)) as json_f:
        json_data = json.load(json_f)
        labels = list(json_data.values())
    return labels

In [ ]:
def parse_json_dict(path):
    """Analyse le fichier d'annotations

    Args:
        path (str): Chemin vers le fichier d'annotations (json)

    Returns:
        dict: Dictionnaire avec la correspondance image: annotation
    """
    fname = glob('*.json', dir_fd=path)[0]
    with open(os.path.join(path, fname)) as json_f:
        labels = json.load(json_f)
    return labels

In [ ]:
def inspect_dataset(path):
    """Analyse le jeu de donné

    Args:
        path (str): Chemin vers le fichier d'annotations (json)
    """
    colors = ['red', 'lime']
    labels = ['train', 'val']
    data_train = parse_json_list(os.path.join(path, 'train'))
    data_val = parse_json_list(os.path.join(path, 'val'))
    bins = np.linspace(min(data_train), max(data_train), max(data_train))
    plt.style.use('ggplot')
    plt.hist((data_train, data_val), bins, color = colors, label = labels)
    plt.legend(prop={'size': 10})
    plt.title('label repartition')
    plt.xlabel('number of boxes on image')
    plt.ylabel('instances')
    plt.show()
    print('moyenne train: ', statistics.mean(data_train))

In [ ]:
def get_padding(image, imgsz):    
    w, h = image.size
    w_padding = max((imgsz[0] - w) / 2, 0)
    h_padding = max((imgsz[1] - h) / 2, 0)
    t_pad = h_padding if h_padding % 1 == 0 else h_padding+0.5
    l_pad = w_padding if w_padding % 1 == 0 else w_padding+0.5
    b_pad = h_padding if h_padding % 1 == 0 else h_padding-0.5
    r_pad = w_padding if w_padding % 1 == 0 else w_padding-0.5
    padding = (int(l_pad), int(t_pad), int(r_pad), int(b_pad))
    return padding

class NewPad(object):
    def __init__(self, fill=0, padding_mode='constant', imgsz = [224,224]):
        assert isinstance(fill, (numbers.Number, str, tuple))
        assert padding_mode in ['constant', 'edge', 'reflect', 'symmetric']
        self.imgsz = imgsz
        self.fill = fill
        self.padding_mode = padding_mode
        
    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be padded.

        Returns:
            PIL Image: Padded image.
        """
        return F.pad(img, get_padding(img, self.imgsz), self.fill, self.padding_mode)
    
    def __repr__(self):
        return self.__class__.__name__ + '(padding={0}, fill={1}, padding_mode={2})'.\
            format(self.fill, self.padding_mode)


In [ ]:
class MyDataset(Dataset):
    def __init__(self, data_dir, split='train', transform=None):
        self.data_dir = os.path.join(data_dir, split)
        self.transform = transform
        self.image_files = [file for file in os.listdir(self.data_dir) if file.endswith('.jpg') or file.endswith('.jpeg')]
        self.labels = self.parse_json()
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        image_path = os.path.join(self.data_dir, self.image_files[idx])
        image = Image.open(image_path).convert("RGB")
        label = self.labels[self.image_files[idx]]
        label = torch.tensor(label, dtype=torch.float32)
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def collate_fn(self, batch):
        images_zip, labels_zip = zip(*batch)
        images_batch = torch.stack(images_zip, dim = 0)
        labels_batch = torch.stack(labels_zip, dim = 0)
        return (images_batch, labels_batch)
    
    def parse_json(self):
        fname = glob('*.json', dir_fd=self.data_dir)[0]
        with open(os.path.join(self.data_dir, fname)) as json_f:
            labels = json.load(json_f)
        return labels


In [ ]:
def train_step(inputs, targets, model, optimizer, criterion):
    """Une étape d'entraînement (un batch)

    Args:
        inputs (torch.Tensor): Batch de données
        targets (torch.Tensor): Valeurs cibles
        model (nn.Module): Modèle de regression
        optimizer (Optimizer): Optimiseur
        criterion (Loss Function): Fonction de perte

    Raises:
        ValueError: Erreur si explosion du gradient

    Returns:
        acc(float): la précision du model, nombre de prédictions exactement juste/ nombre de prédictions
        loss.item()(float): valeur de perte (décalage entre prédiction et réalité), dépend de la fonction choisie (valeur absolue, quadratique, huber)
        mae.item()(float): valeur de perte (fonction MAE) c'est la valeur abolue de décalage
    """
    device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mae_loss = nn.L1Loss()
    model.train()
    optimizer.zero_grad()
    inputs = inputs.to(device)
    targets = targets.to(device)
    try:
        predictions = model(inputs)
        loss = criterion(predictions.squeeze(), targets)
        mae = mae_loss(predictions.squeeze(), targets)
        loss.backward()
        optimizer.step()
        acc = 0
        for i in range(len(predictions)):
            pred = round(predictions[i].item())
            target = int(targets[i].item())
            if pred == target:
                acc +=1
        acc /= len(predictions)
    except ValueError as ve:
        raise ValueError(ve)
    
    # nettoyage avant sortie
    del inputs, targets
    torch.cuda.empty_cache()
    return acc, loss.item(), mae.item()


In [ ]:
def val_step(inputs, targets, model, criterion):
    """Une étape d'entraînement (un batch)

    Args:
        inputs (torch.Tensor): Batch de données
        targets (torch.Tensor): Valeurs cibles
        model (nn.Module): Modèle de regression
        criterion (Loss Function): Fonction de perte

    Raises:
        ValueError: Erreur si explosion du gradient

    Returns:
        acc(float): la précision du model, nombre de prédictions exactement juste/ nombre de prédictions
        loss.item()(float): valeur de perte (décalage entre prédiction et réalité), dépend de la fonction choisie (valeur absolue, quadratique, huber)
        mae.item()(float): valeur de perte (fonction MAE) c'est la valeur abolue de décalage
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mae_loss = nn.L1Loss()
    model.eval()
    inputs = inputs.to(device)
    targets = targets.to(device)
    acc = 0
    try:
        with torch.no_grad():
            predictions = model(inputs)
            loss = criterion(predictions.squeeze(), targets)
            mae = mae_loss(predictions.squeeze(), targets)
            for i in range(len(predictions)):
                pred = round(predictions[i].item())
                target = int(targets[i].item())
                if pred == target:
                    acc +=1
        acc /= len(predictions)
    except ValueError as ve:
        raise ValueError(ve)
    
    # nettoyage avant sortie
    del inputs, targets
    torch.cuda.empty_cache()
    return acc, loss.item(), mae.item()

In [ ]:
def get_confusion_matrix(model, path, transform):
    """Donne la matrice de confusion

    Args:
        model (nn.Module): Modèle de regression
        path (str): Chemin vers le fichier d'annotations
        transform (torchvision.transforms): Outil de pré-traitement des images

    Returns:
        np.ndarray: Matrice de confusion
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    fnames= glob('*.jpg', dir_fd = path)
    labels = parse_json_dict(path)
    # init matrix
    indices_t = np.linspace(1, max(labels.values()), max(labels.values()))
    indices_p = np.linspace(1, 40, 40)
    confusion_matrix = np.zeros((indices_p.size+1, indices_t.size), dtype = np.int16)
    # predictions
    model.eval()
    for fname in fnames:
        image = Image.open(os.path.join(path, fname)).convert("RGB")
        image = transform(image)
        image = torch.unsqueeze(image, dim=0)
        image = image.to(device)
        pred = model(image)
        pred = round(pred[0][0].item())
        if pred in indices_p:
            confusion_matrix[pred-1][labels[fname]-1]+=1
        else:
            confusion_matrix[-1][labels[fname]-1]+=1
    return confusion_matrix, indices_p, indices_t

In [ ]:
def training_loop(train_params):
    """Entraine le modèle

    Args:
        train_params (dict): Dictionnaire contenant les hyperparamètres sassociées à l'entrainement

    Do:
        Sauvegarde le meilleur modèle "best.pt"
        Sauvegarde l'évolution des résultats "record.tsv" et son graphe "results.png"
        Sauvegarde les résultats finaux "resultats.yaml"
        Génère et sauvegarde les matrice de confusions (train, val)

    Returns:
        float: La valeur de la meilleure précision (des meilleurs poids)
    """
    model = train_params['model']
    num_epochs = train_params['epochs']
    earlystop = train_params['earlystop']
    results_dir = train_params['results_dir']
    dataloaders = train_params['dataloarders']
    optimizer = train_params['optimizer']
    criterion = train_params['criterion']
    data_transforms = train_params['data_transforms']
    print(Fore.CYAN + results_dir)
    print(os.path.join(os.getcwd(), results_dir) + Fore.RESET)
    # initialisations
    average_training_losses = []
    average_training_accs = []
    average_val_accs = []
    average_val_losses = []
    average_training_maes = []
    average_val_maes = []
    best_val_accs = 0
    best_epoch = 0
    early_stopping = None

    start_time = time.time()
    with open(os.path.join(results_dir, 'record.tsv'), 'w') as tsvfile:
        writer = csv.writer(tsvfile, delimiter = '\t', lineterminator = '\n')
        writer.writerow(['epoch', 'train_loss', 'train_mae_loss', 'train_acc', 'val_loss', 'val_mae_loss', 'val_acc'])
        for epoch in range(num_epochs):
            training_losses = []
            training_accs = []
            training_maes = []
            desc = 'Epoch ' + str(epoch) + '/' + str(num_epochs)
            for (x_train, y_train) in tqdm(dataloaders['train'], desc = desc):
                try:
                    acc, loss, mae = train_step(x_train, y_train, model, optimizer, criterion)
                except ValueError as ve:
                    print(Fore.RED + ve + Fore.RESET)
                    yaml_dict = {'error': str(ve) + ' during train: exploding gradient!'}
                    with open(os.path.join(results_dir, 'results.yaml'), 'w') as yamlf:
                        yaml.dump(yaml_dict, yamlf, default_flow_style=False, allow_unicode=True)
                    return 0
                    
                training_losses.append(loss)
                training_accs.append(acc)
                training_maes.append(mae)
            average_training_loss = sum(training_losses) / len(training_losses)
            average_training_acc = sum(training_accs) / len(training_accs)
            average_training_mae = sum(training_maes) / len(training_maes)
            average_training_losses.append(average_training_loss)
            average_training_accs.append(average_training_acc)
            average_training_maes.append(average_training_mae)

            # Evaluation
            val_accs = []
            val_loss = []
            val_maes = []
            for x_val, y_val in dataloaders['val']:
                try:
                    acc, loss, mae = val_step(x_val, y_val, model, criterion)
                except ValueError as ve:
                    print(Fore.RED + ve + Fore.RESET)
                    yaml_dict = {'error': str(ve) + ' during val: exploding gradient!'}
                    with open(os.path.join(results_dir, 'results.yaml'), 'w') as yamlf:
                        yaml.dump(yaml_dict, yamlf, default_flow_style=False, allow_unicode=True)
                    return 0
                
                val_accs.append(acc)
                val_loss.append(loss)
                val_maes.append(mae)
            average_val_loss = sum(val_loss) / len(val_loss)
            average_val_acc = sum(val_accs) / len(val_accs)
            average_val_mae = sum(val_maes)/len(val_maes)
            average_val_accs.append(average_val_acc)
            average_val_losses.append(average_val_loss)
            average_val_maes.append(average_val_mae)
            if best_val_accs < average_val_acc:
                torch.save(model.state_dict(), os.path.join(results_dir, 'best.pt'))
                best_val_accs = average_val_acc
                best_epoch = epoch
                print(Fore.GREEN + "New best model: ", best_val_accs, Fore.RESET)
            # Early stopping
            if earlystop is not None:
                if epoch > best_epoch+earlystop:
                    early_stopping = epoch
                    break
            writer.writerow([epoch, average_training_loss, average_training_mae, average_training_acc, average_val_loss, average_val_mae, average_val_acc])

    end_t = time.time()
    time_elapsed=end_t - start_time
    legend = ['train', 'val']
    print("time_elapsed: {}".format(str(datetime.timedelta(seconds=time_elapsed))))
    plt.style.use('dark_background')

    fig = plt.figure(layout="constrained", figsize=(20,10))
    gs = GridSpec(3, 1, figure=fig, wspace = 0.1)
    fig.suptitle('Results', fontsize=16)

    ax = fig.add_subplot(gs[0])
    intervals = np.arange(len(average_training_losses))
    ax.plot(intervals, average_training_losses, color = 'b')
    intervals = np.arange(1,len(average_training_losses))
    ax.plot(intervals, average_val_losses[1:], color = 'g')
    ax.set_ylabel("loss: " + criterion._get_name())

    ax = fig.add_subplot(gs[1])
    intervals = np.arange(len(average_training_accs))
    ax.plot(intervals, average_training_maes, color = 'b')
    intervals = np.arange(1,len(average_training_accs))
    ax.plot(intervals, average_val_maes[1:], color = 'g')
    ax.set_ylabel("loss: MAE")

    ax = fig.add_subplot(gs[2])
    intervals = np.arange(len(average_training_accs))
    ax.plot(intervals, average_training_accs, color = 'b')
    ax.plot(intervals, average_val_accs, color = 'g')
    ax.set_xlabel("epoch")
    ax.set_ylabel("accuracy")

    fig.legend(legend)
    fig.savefig(os.path.join(results_dir, 'results.png'))
    plt.close()
    if best_val_accs == 0 :
        print("Evaluation accuracy (best) = ", best_val_accs)
    elif best_val_accs > 0.9:
        print(Back.GREEN + "Evaluation accuracy (best) = ", best_val_accs)
    elif best_val_accs > 0.8:
        print(Back.BLUE + "Evaluation accuracy (best) = ", best_val_accs)
    elif best_val_accs > 0.65:
        print('\033[48;2;255;90;0m' + "Evaluation accuracy (best) = ", best_val_accs)
    else:
        print(Back.RED + "Evaluation accuracy (best) = ", best_val_accs)
    print(Back.RESET)

    # clear GPU cache memory
    torch.cuda.empty_cache()

    # Sauvegarde des résultats
    yaml_dict = {'best epoch': best_epoch,
                'best val acc': best_val_accs,
                'train loss': average_training_losses[best_epoch],
                'train acc': average_training_accs[best_epoch],
                'train mae loss': average_training_maes[best_epoch],
                'val loss': average_val_accs[best_epoch],
                'val mae loss': average_val_maes[best_epoch],
                'early_stopping': early_stopping,
                'time': str(datetime.timedelta(seconds=time_elapsed))}
    with open(os.path.join(results_dir, 'results.yaml'), 'w') as yamlf:
        yaml.dump(yaml_dict, yamlf, default_flow_style=False, allow_unicode=True)

    # Sauvegarde des matrices de confusion
    path = os.path.join('dataset', 'val')
    conf_mat, inds_p, inds_t = get_confusion_matrix(model, path, data_transforms)
    inds_p = np.append(inds_p, 'other')
    df_cm = pd.DataFrame(conf_mat, index = [i for i in inds_p],
                    columns = [i for i in inds_t])
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)
    plt.ylabel('predicted')
    plt.xlabel('ground truth')
    plt.title('Val confusion matrix')
    plt.savefig(os.path.join(results_dir, 'val_conf_mat.png'))
    plt.close()

    # train
    path = os.path.join('dataset', 'train')
    conf_mat, inds_p, inds_t = get_confusion_matrix(model, path, data_transforms)
    inds_p = np.append(inds_p, 'other')
    df_cm = pd.DataFrame(conf_mat, index = [i for i in inds_p],
                    columns = [i for i in inds_t])
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)
    plt.ylabel('predicted')
    plt.xlabel('ground truth')
    plt.title('train confusion matrix')
    plt.savefig(os.path.join(results_dir, 'train_conf_mat.png'))
    plt.close()

    return best_val_accs


In [ ]:
def get_criterion(params):
    """Donne la fonction de perte (instance de la classe Loss)

    Args:
        params (str): Nom de la fonction

    Returns:
        criterion: Instance de la fonction de perte
    """
    match params['loss_function']:
        case 'MSE':
            criterion = nn.MSELoss()
        case 'MAE':
            criterion = nn.L1Loss()
        case 'Huber':
            criterion = nn.HuberLoss(delta = params['delta'])
    return criterion

In [ ]:
def get_optimizer(model, params):
    """Donne l'optimiseur (instance de la classe Optimizer)

    Args:
        model (nn.Module): Modèle
        params (dict): Hyperparamètres

    Returns:
        optimizer: Instance de l'optimiseur
    """
    lr = params['lr']
    wd = params['weight_decay']
    eps = params['epsilon']
    rho = params['rho']
    init = params['initial_accumulator_value']
    betas = params['betas']
    max_iter = params['max_iter']
    md = params['momentum_decay']
    alpha = params['alpha']
    momentum = params['momentum']
    
    match params['optimizer']:
        case 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
        case 'Adadelta':
            optimizer = torch.optim.Adadelta(model.parameters(), lr = lr, rho = rho, eps=eps, weight_decay=wd)
        case 'Adagrad':
            optimizer = torch.optim.Adagrad(model.parameters(), lr=lr, weight_decay=wd, eps=eps, initial_accumulator_value=init)
        case 'RMSprop':
            optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, weight_decay=wd, eps=eps, alpha=alpha, momentum=momentum)
        case 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd, eps=eps, betas=betas)
        case 'AdamW':
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, eps=eps, betas=betas)
        case 'LBFGS':
            optimizer = torch.optim.LBFGS(model.parameters(), lr=lr, max_iter=max_iter)
        case 'NAdam':
            optimizer = torch.optim.NAdam(model.parameters(), lr=lr, weight_decay=wd, eps=eps, betas=betas, momentum_decay=md)
        case 'RAdam':
            optimizer = torch.optim.RAdam(model.parameters(), lr=lr, weight_decay=wd, eps=eps, betas=betas)
    return optimizer

In [ ]:
def get_dataloaders(params):
    """Donne les Dataloaders (instance de la classe Dataloader)

    Args:
        params (dict): Dictionnaire des hyperparamètres

    Returns:
        dataloaders: Les dataloaders des differents splits (test, val,...)
        data_transforms: Outils de pré-traitement des données
    """
    data_dir = params['dataset']
    batch_size = params['batch_size']
    imgsz  = params['imgsz']
    dataloaders = {}
    splits = []
    for split in ['test', 'val', 'train']:
        if split in os.listdir(data_dir):
            splits .append(split)

    data_transforms = transforms.Compose([
            NewPad(imgsz = imgsz),
            transforms.Resize((imgsz[1], imgsz[0])), # h,w
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])           
     
    for split in splits:
        dataset = MyDataset(data_dir = data_dir, split = split, transform = data_transforms)
        dataloaders[split] = DataLoader(dataset=dataset, shuffle=True, batch_size = batch_size, collate_fn = dataset.collate_fn)
    return dataloaders, data_transforms

In [ ]:
def Test_set(params):
    """Prépare et lance l'entrainement d'un modèle

    Args:
        params (dict): Dictionnaire des hyperparamètres

    Do:
        Sauvegarde les hyperparamètres "args.yaml"
        Sauvegarde l'architecture du modèle "model.txt"

    Returns:
        float: Meilleur précision du meilleur modèle
    """
    if os.path.exists(os.path.join(params['results_dir'], 'results.yaml')):
        print('Already trained')
        with open(os.path.join(params['results_dir'], 'results.yaml'), 'r') as res:
            res_data = yaml.safe_load(res)
            try:
                best_val_acc = res_data['best val acc']
            except KeyError:
                best_val_acc = 0
            return best_val_acc
    os.makedirs(params['results_dir'], exist_ok=True)
    # Sauvegarde des paramètres
    with open(os.path.join(params['results_dir'], 'args.yaml'), 'w') as yamlf:
        yaml.dump(params, yamlf, default_flow_style=False, allow_unicode=True)

    # Génération du modèle
    model = gen_new_model(params['freeze'])
    device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device) # déplacer le model vers le GPU a faire avant de construire l'optimiseur (pour adagrad)
    optimizer = get_optimizer(model, params)
    criterion = get_criterion(params)
    dataloaders, data_transforms = get_dataloaders(params)
    
    # Sauvegarde de l'architecture du modèle
    model_stats = summary(model, input_size=(params['batch_size'], 3, 224, 224), row_settings=("depth", "ascii_only"))
    summary_str = str(model_stats)
    with open(os.path.join(params['results_dir'], 'model.txt'), 'w') as modelf:
        modelf.write(summary_str)

    # entrainement
    train_params = {
        'model': model,
        'epochs': params['epochs'],
        'earlystop': params['earlystop'],
        'results_dir': params['results_dir'],
        'dataloarders': dataloaders,
        'optimizer': optimizer,
        'criterion': criterion,
        'data_transforms': data_transforms
    }
    best_acc = training_loop(train_params)
    return best_acc

In [ ]:
def GridSchearch(grid):
    """Fait la recherche de grille (entraine un modèle pour chaque combinaison d'hyperparamètres)

    Args:
        grid (list: dict): Liste de dictionnaire correspondants aux combinaisons d'hyperparamètres

    Returns:
        best_params(dict): Dictionnaire avec les meilleurs hyperparamètres
        best_acc(float): Meilleure précision pour la meilleure combinaisonn d'hyperparamètres
        results(dict): Dictionnaire avec la meilleure précision pour chaque combinaison (chemin/des/resultats/de/la/combinaison: meilleure précision)
    """
    # init
    results = [('Path', 'Best Accuracy')]
    best_acc = 0
    best_params = {'None': None}
    i = 0 # seulement pour l'affichage
    for params_set_id in range(len(grid)):
        i+=1
        if i%5==0:
            clear_output() # otherwise too many outputs
        params_set = grid[params_set_id]
        print(Fore.RED + '=========================================================={}/{}=========================================================='.format(params_set_id, len(grid)))
        res = Test_set(params_set)
        results.append((os.path.join(os.getcwd(), params_set['results_dir']), res))
        if res>best_acc:
            best_acc = res
            best_params = params_set
    best_params['best_acc'] = best_acc
    return best_params, best_acc, results

In [ ]:
def gen_pdf(results, path, best):
    """Génère un pdf avec simplement un tableau récapitulatif des résultats pour chaque combinaison

    Args:
        results (dict): Resultats de la recherche en grille
        path (str): Chemin + nom du fichier pdf
        best (float): Valeur de la meilleure précision du meilleur modèle
    """
    # create document
    pdf = Document()
    
    # add page
    page = Page(Decimal(1500), Decimal(1684))
    m = Decimal(10)
    pdf.add_page(page)
    layout = SingleColumnLayoutWithOverflow(page)
    colors = {0: X11Color("Red"),
                0.65: X11Color("Orange"),
                0.8: X11Color("Blue"),
                0.90: X11Color("Green")}
    for i in range(0, len(results), 38):
        res = results[i:min(i+38, len(results))]
        table = FlexibleColumnWidthTable(number_of_rows=len(res), number_of_columns=2)
        for name, v in [(k,v) for k,v in res]:
            if isinstance(v, str):
                table.add(TableCell(Paragraph(text=name, font="Helvetica-Bold", font_size=Decimal(12))))
                table.add(TableCell(Paragraph(text=v, font="Helvetica-Bold", font_size=Decimal(12))))
                continue
            table.add(TableCell(Paragraph(name)))
            c = X11Color("Black")
            for b,bc in colors.items():
                if v > b:
                    c = bc
                if v == best:
                        c = X11Color('Purple')
            table.add(            TableCell(
                    Paragraph(str(v), horizontal_alignment=Alignment.CENTERED),
                    background_color=c
                ))
        table.set_padding_on_all_cells(Decimal(m), Decimal(m), Decimal(m), Decimal(m))
        # set border
        table.set_border_width_on_all_cells(Decimal(0.2))
        layout.add(table)
    with open(path, "wb") as in_file_handle:
        PDF.dumps(in_file_handle, pdf)

In [ ]:
def main():
    """Génère la grille de recherche.
    Appelle la fonction de recherche.
    Génère le pdf de bilan.
    """
    grid = make_grid()
    best_params, best_acc, results = GridSchearch(grid)
    print(Style.RESET_ALL)
    print(Back.MAGENTA + f'Best accuracy from run: {best_acc}' + Back.RESET)
    print(Fore.MAGENTA + 'Params:',best_params, Fore.RESET)
    gen_pdf(results, os.path.join(grid_results_dir, results_pdf), best_params['best_acc'])
    return

In [ ]:
main()